In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv("/Users/hariharprasad/MyDocuments/Code/Python/MNIST with NumPy/Datasets/mnist_train.csv")

In [ ]:
data = np.array(data) 
m,n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T #transposing the data to have one image per row, and a label row instead of a label column
Y_dev = data_dev[0] #array of all the labels of the testing data
X_dev = data_dev[1:n] #array of all the actual pixel values of the testing data
X_dev = X_dev/255. #normalising the data from a range of 0 to 255 to 0 to 1

data_train = data[1000:m].T #transposing the data to have one image per row, and a label row instead of a label column
Y_train = data_train[0] #array of all the labels of the training data 
X_train = data_train[1:n] #array of all the actual pixel values of the training data
X_train = X_train / 255. #normalising the data from a range of 0 to 255 to 0 to 1

In [ ]:
def init_params(): #initiating all weights and biases with a value between -0.5 and 0.5
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10,1) - 0.5
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z): #implementing the Rectified Linear Unit function to return Z if Z > 0, and 0 otherwise
    return np.maximum(Z,0)

def ReLU_deriv(Z): #returning the derivative of the ReLU function, which is 0 if Z is less than or equal to 0, or 1 otherwise (graph of ReLU is just a straight line)
    return Z > 0

def softmax(Z): #softmax function to be implemented for normalisation of output layer between 0 and 1. e^one_value/e^every value
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def one_hot(Y): #converting labels to one hot encoded array between 0 to 1, with the corresponding index being 1 and all other indexes being 0
     one_hot_Y = np.zeros((Y.size, Y.max() + 1))
     one_hot_Y[np.arange(Y.size), Y] = 1
     one_hot_Y = one_hot_Y.T
     return one_hot_Y

def forward_prop(W1, b1, W2, b2, X): #forward propagation, getting values with the random initial weights and biases
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y): #backward propagation, modifying the weights and biases to minimise the loss function
    one_hot_y = one_hot(Y)
    dZ2 = A2 - one_hot_y
    dW2 = 1/m*dZ2.dot(A1.T)
    db2 = 1/m*np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1/m*dZ1.dot(X.T)
    db1 = 1/m*np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): #updating the parameters to reduce the loss function
    W1 = W1 - alpha*dW1
    b1 = b1 - alpha * db1
    W2 = W2 -alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2): #getting predictions from network
    return np.argmax(A2,0) #getting index with highest value across 0th axis 

def get_accuracy(predictions, Y): #calculating accuracy of predictions
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations): #calling all the functions in order to perform gradient descent
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500) #setting learning rate and number of iterations

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_dev[:, index, None]
    prediction = make_predictions(current_image, W1, b1, W2, b2)
    label = Y_dev[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation = "nearest")
    plt.show()

test_prediction(2, W1, b1, W2, b2)